# 01. Importing libraries

In [25]:
import pandas as pd
import numpy as np
import os

# 02. Importing data

In [26]:
path = r'/Users/marcela/Achievement 4'

In [27]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_busiestdays.pkl'))

# 03. Grouping data

In [28]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


## While the subset provides insights into individual sectors, the full dataset offers the mean across all departments.

# 04. Creating a loyalty flag

In [29]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform('max')

In [47]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [31]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [32]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [33]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

# 05. Checking the basic statistics of the product prices for each loyalty category

In [34]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.388747  1.0  99999.0
New customer      13.294370  1.0  99999.0
Regular customer  12.496203  1.0  99999.0

# 06. Creating a spending flag for each user based on the average price across all their orders using the following criteria:
## If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
## If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [35]:
ords_prods_merge.groupby('user_id').agg({'prices': ['mean']})

,prices
,mean
user_id,
1,6.367797
2,7.515897
3,8.197727
4,8.205556
5,9.189189
...,...
206205,8.909375
206206,7.646667


In [36]:
ords_prods_merge['spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [48]:
ords_prods_merge.loc[ords_prods_merge['spending'] < 10, 'spending_flag'] = 'low spender'

In [38]:
ords_prods_merge.loc[ords_prods_merge['spending'] >= 10, 'spending_flag'] = 'high spender'

In [39]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
low spender     31798751
high spender      635461
Name: count, dtype: int64

# 07. Creating an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
## If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
## If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
## If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [40]:
ords_prods_merge.groupby('user_id').agg({'days_since_prior_order': ['median']})

,days_since_prior_order
,median
user_id,
1,20.5
2,13.0
3,10.0
4,20.0
5,11.0
...,...
206205,30.0
206206,3.0


In [41]:
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [49]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [43]:
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] <= 20) & (ords_prods_merge['order_frequency'] > 10), 'order_frequency_flag'] = 'Regular customer'

In [44]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [45]:
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: count, dtype: int64

In [50]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_grouping.pkl'))